In [9]:
import os
import torch
from torchvision import datasets
from torchvision.transforms import Compose, Resize, ToTensor
from torch.utils.data import DataLoader, ConcatDataset
import numpy as np

## Étape 1 : Chargement des datasets

In [10]:
dataset_root = "ipeo_hurricane_for_students"

# Transform simple : juste resize et ToTensor (pas de normalisation)
transform = Compose([
    Resize((224, 224)),
    ToTensor()
])

# Charger SEULEMENT le training set pour calculer les stats
# (Bonne pratique : ne pas utiliser val/test pour éviter data leakage)
print("Chargement du TRAINING SET SEULEMENT...")
train_ds = datasets.ImageFolder(os.path.join(dataset_root, "train"), transform=transform)

print(f"Train : {len(train_ds)} images")
print(f"\n⚠️  Note: Nous calculons les statistiques SEULEMENT sur le training set")
print(f"   pour éviter la fuite d'information (data leakage).")
print(f"   Ces statistiques seront ensuite appliquées à val et test sets.")


Chargement du TRAINING SET SEULEMENT...
Train : 19000 images

⚠️  Note: Nous calculons les statistiques SEULEMENT sur le training set
   pour éviter la fuite d'information (data leakage).
   Ces statistiques seront ensuite appliquées à val et test sets.


## Étape 2 : Calcul des statistiques

In [11]:
# Créer un DataLoader avec SEULEMENT le training set
batch_size = 32
loader = DataLoader(train_ds, batch_size=batch_size, shuffle=False, num_workers=0)

print("Calcul des statistiques sur le TRAINING SET SEULEMENT...\n")

# Accumulator pour mean et std
mean_accumulator = torch.zeros(3)
std_accumulator = torch.zeros(3)
total_pixels = 0

# Première passe : calculer la moyenne
print("Passe 1 : calcul de la moyenne...")
for batch_idx, (images, _) in enumerate(loader):
    # images shape: (batch_size, 3, 224, 224)
    batch_size_actual = images.shape[0]
    height, width = images.shape[2], images.shape[3]
    
    # Remodeler : (batch_size * height * width, 3)
    images_reshaped = images.permute(0, 2, 3, 1).reshape(-1, 3)
    
    # Accumuler
    mean_accumulator += images_reshaped.sum(dim=0)
    total_pixels += images_reshaped.shape[0]
    
    if (batch_idx + 1) % 5 == 0:
        print(f"  Batch {batch_idx + 1}/{len(loader)}")

# Moyenne globale
mean = mean_accumulator / total_pixels
print(f"\nMoyenne calculée : {mean}")

# Deuxième passe : calculer l'écart-type
print("\nPasse 2 : calcul de l'écart-type...")
total_pixels = 0
for batch_idx, (images, _) in enumerate(loader):
    # Remodeler pour le calcul
    images_reshaped = images.permute(0, 2, 3, 1).reshape(-1, 3)
    
    # Calculer la variance
    diff_squared = (images_reshaped - mean.unsqueeze(0)) ** 2
    std_accumulator += diff_squared.sum(dim=0)
    total_pixels += images_reshaped.shape[0]
    
    if (batch_idx + 1) % 5 == 0:
        print(f"  Batch {batch_idx + 1}/{len(loader)}")

# Écart-type global
std = torch.sqrt(std_accumulator / total_pixels)

print(f"\nÉcart-type calculé : {std}")


Calcul des statistiques sur le TRAINING SET SEULEMENT...

Passe 1 : calcul de la moyenne...
  Batch 5/594
  Batch 10/594
  Batch 15/594
  Batch 20/594
  Batch 25/594
  Batch 30/594
  Batch 35/594
  Batch 40/594
  Batch 45/594
  Batch 50/594
  Batch 55/594
  Batch 60/594
  Batch 65/594
  Batch 70/594
  Batch 75/594
  Batch 80/594
  Batch 85/594
  Batch 90/594
  Batch 95/594
  Batch 100/594
  Batch 105/594
  Batch 110/594
  Batch 115/594
  Batch 120/594
  Batch 125/594
  Batch 130/594
  Batch 135/594
  Batch 140/594
  Batch 145/594
  Batch 150/594
  Batch 155/594
  Batch 160/594
  Batch 165/594
  Batch 170/594
  Batch 175/594
  Batch 180/594
  Batch 185/594
  Batch 190/594
  Batch 195/594
  Batch 200/594
  Batch 205/594
  Batch 210/594
  Batch 215/594
  Batch 220/594
  Batch 225/594
  Batch 230/594
  Batch 235/594
  Batch 240/594
  Batch 245/594
  Batch 250/594
  Batch 255/594
  Batch 260/594
  Batch 265/594
  Batch 270/594
  Batch 275/594
  Batch 280/594
  Batch 285/594
  Batch 290/594


## Résultats

In [12]:
print("="*70)
print("STATISTIQUES DU TRAINING SET (Bonne pratique)")
print("="*70)
print(f"\nMean par canal (RGB) :")
print(f"  {mean.tolist()}")
print(f"\nStandard Deviation par canal (RGB) :")
print(f"  {std.tolist()}")

print(f"\n\n" + "="*70)
print("CODE À COPIER DANS VOTRE NOTEBOOK")
print("="*70)
print(f"""
import torch
import torchvision.transforms as T

# Statistiques calculées sur le TRAINING SET UNIQUEMENT
# (Évite la fuite d'information et simule la réalité)
mean = torch.tensor({mean.tolist()})
std = torch.tensor({std.tolist()})

normalize = T.Normalize(mean, std)

# Transform SANS augmentation
no_augmentation_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    normalize
])

# Transform AVEC augmentation
augmented_transform = T.Compose([
    T.Resize((224, 224)),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(degrees=15),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    T.ToTensor(),
    normalize
])

# Appliquer les mêmes statistiques à train, val, et test
train_ds = datasets.ImageFolder(base_dir + '/train', transform=your_chosen_transform)
val_ds = datasets.ImageFolder(base_dir + '/validation', transform=no_augmentation_transform)
test_ds = datasets.ImageFolder(base_dir + '/test', transform=no_augmentation_transform)
""")

print("\n📌 IMPORTANT:")
print("   - Les statistiques (mean/std) sont calculées sur le TRAINING SET seulement")
print("   - Ces mêmes statistiques sont appliquées à validation et test sets")
print("   - Cela évite la fuite d'information (data leakage)")
print("   - C'est plus réaliste : en production, vous calculez les stats sur")
print("     vos données d'entraînement, puis les appliquez à de nouvelles données")


STATISTIQUES DU TRAINING SET (Bonne pratique)

Mean par canal (RGB) :
  [0.3520655930042267, 0.3834735155105591, 0.28522437810897827]

Standard Deviation par canal (RGB) :
  [0.14353738725185394, 0.12388758361339569, 0.12524260580539703]


CODE À COPIER DANS VOTRE NOTEBOOK

import torch
import torchvision.transforms as T

# Statistiques calculées sur le TRAINING SET UNIQUEMENT
# (Évite la fuite d'information et simule la réalité)
mean = torch.tensor([0.3520655930042267, 0.3834735155105591, 0.28522437810897827])
std = torch.tensor([0.14353738725185394, 0.12388758361339569, 0.12524260580539703])

normalize = T.Normalize(mean, std)

# Transform SANS augmentation
no_augmentation_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    normalize
])

# Transform AVEC augmentation
augmented_transform = T.Compose([
    T.Resize((224, 224)),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(degrees=15),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    T.